# CA4 - Question 1

In [ ]:
import re

class ValidatorMeta(type):
    def __new__(cls, name, bases, dct):
        
        validation_rules = {
            #TO DO
        }

        
        custom_error_messages = {
            'email': 'Email must have a valid format and end with .com or .org.',
            'phone_number': 'Phone number must start with +98 and be followed by 8 digits.',
            'password': 'Password must be between 8 and 12 characters and include at least one uppercase letter, one lowercase letter, one number, and one special character (*, @, %, !).',
            'product_code': 'Product code must consist of 2 uppercase letters, 2 to 4 digits, an optional lowercase letter, and an optional version number (v1-v99).',
            'stop_word': 'The word "Stop" or "stop" must be separate and not part of a larger word or attached to punctuation.',
            'repeated_phrase': 'The phrase "some students" or "many employees" must repeat exactly, with no different terms in between.',
            'date': 'Date must be in YYYY/MM/DD format, with valid month and day values.',
            'quotation': 'Text must be enclosed in balanced single or double quotes and contain only letters, numbers, and spaces.',
            'parenthesis': 'Parentheses must be balanced with no unmatched opening or closing parentheses.',
        }

        # Method generator for validation functions
        def create_validator(field, rule, custom_message):
            def validator(self, value):
                if not re.match(rule, value):
                    raise ValueError(custom_message)
                return True
            return validator

        # Add validation methods to the class dynamically
        for field, rule in validation_rules.items():
            dct[f'validate_{field}'] = create_validator(field, rule, custom_error_messages.get(field, f"Invalid {field}"))

        return super().__new__(cls, name, bases, dct)


class FormValidator(metaclass=ValidatorMeta):
    """Concrete validator class with all validation methods"""
    pass


In [3]:
validator = FormValidator()

# Test Validation
# TO DO
